In [ ]:

import pinecone
from pinecone import ServerlessSpec
from sentence_transformers import SentenceTransformer
import numpy as np
import time

# Initialize Pinecone with API key
PINECONE_API_KEY = "PINECONE_API_KEY"
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
index_name = "meta-data"
index = pc.Index(index_name)
# Load the sentence transformer model for creating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define different namespaces
namespaces = {
    "quaid_azam": [
        {"text": "Quaid-e-Azam was the founder of Pakistan.", "author": "Quaid-e-Azam", "year": 1947}
    ],
    "allama_iqbal": [
        {"text": "Allama Iqbal was a great poet and philosopher.", "author": "Allama Iqbal", "year": 1930}
    ],
    "imran_khan": [
        {"text": "Imran Khan led Pakistan to a World Cup victory in 1992.", "author": "Imran Khan", "year": 1992}
    ],
    "sir_sayyed": [
        {"text": "Sir Syed Ahmad Khan was a reformer and educator.", "author": "Sir Syed Ahmad Khan", "year": 1875}
    ]
}

def store_data():
    """Stores data with metadata in Pinecone under different namespaces."""
    for namespace, items in namespaces.items():
        for item in items:
            embedding = model.encode(item["text"]).astype(np.float32)
            vector_id = str(abs(hash(item["text"])))[:16]
            metadata = {"text": item["text"], "author": item["author"], "year": item["year"]}
            index.upsert(vectors=[(vector_id, embedding.tolist(), metadata)], namespace=namespace)
    return "Data stored successfully."

def filter_data(namespace):
    """Filters data based on namespace."""
    results = index.query(vector=[0] * 384, top_k=10, include_metadata=True, namespace=namespace)
    return results

if __name__ == "__main__":
    store_data()
    query_namespace = "allama_iqbal"  # Example namespace filter
    filtered_results = filter_data(query_namespace)
    for match in filtered_results.get("matches", []):
        print(f"Text: {match['metadata']['text']}, Author: {match['metadata']['author']}, Year: {match['metadata']['year']}")


c:\Users\Smart Computer\anaconda3\envs\myenv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Text: Allama Iqbal was a great poet and philosopher., Author: Allama Iqbal, Year: 1930.0
Text: Allama Iqbal was a great poet and philosopher., Author: Allama Iqbal, Year: 1930.0
